# Dictionary compilation from Για να κ̇οντούμε γρούσσα νάμου

This notebook aims to create a dictionary from the words contained in Ioannis Kambysis' book, _Για να κχοντούμε γρούσσα νάμου_ (2020) using Google Lens OCR.

# Preparation

In [2]:
# Data wrangling
import pandas as pd
import numpy as np

# Others
import re
import pyperclip as pc
import unicodedata
import copy
import os

# Set clean text

In [18]:
page = 38

In [24]:
text = """
Το κρίε - Τα κρίατα ; Το κρέας - Τα κρέατα
Το γα ; Το γάλα
Α αφόκιουρη ; Η μυζήθρα
Το άλητε - Τα άλητα ; Το αλεύρι - Τα αλεύρια
Οι κουλίντζου ; Είδος ζυμαρικού
Α κολιούρα ; Πίτα που ψήνεται στην πλάνη
Α γριτσ̇έα - Οι γριτσέλε ; Η κουλούρα - Οι κουλούρες
Το γιοργούκι ; Το γιαούρτι
Το άι - Τα άζα ; Το λάδι - Τα λάδια
Το κάζ̌υ - Τα κάζ̌α ; Το καρύδι - Τα καρύδια
Το νίγδαλε - Τα νίγδα ; Το μύγδαλο - Τα μύγδαλα
Ο κούε - Οι κούν̇οι ; Ο σκύλος - Οι σκύλοι
Το κουνάρι - Τα κουνάρια ; Το σκυλάκι - Τα σκυλάκια
Α κατσούα - Οι κατσούλε ; Η γάτα - Οι γάτες
Α κότ̇α - Οι κότε ; Η κότα - Οι κότες
Ο βούλε - Οι βούλοι ; Ο κόκορας - Οι κοκόροι
Ο όνε - Οι όνου ; Ο γάιδαρος - Οι γαϊδάροι
Το μονάρι - Τα μονάρια ; Το μουλάρι - Τα μουλάρια 
Το άγο - Τα άγια  ; Το άλογο - Τα άλογα
Ο βου - Οι βούε ; Το βόδι - Τα βόδια
Ο μόκ̇ο - Οι μότσοι ; Το μοσχάρι - Τα μοσχάρια 
Α κούλικα - Οι κούλιτσ̇ε ; Η αγελάδα - Οι αγελάδες
Το προύατε - Τα προύατα  ; Το πρόβατο - Τα πρόβατα
Ο βάνε - Οι βάνοι ; Το αρνί - Τα αρνιά
Α αιγίδα - Οι αιγίδε  ; Η κατσίκα - Οι κατσίκες
Ο έριφο - Οι ερίφοι ; Το κατσίκι - Τα κατσίκια
Το ρουφάλι - Τα ρουφάλια ; Το κατσικάκι - Τα κατσικάκια 
Ο τσ̌άο - Οι τσ̌άουνε ; Το τραγί - Τα τραγιά
"""

In [23]:
# Save cleaned text
with open(rf'./cleaned_pages/page_{page}.txt', 'w', encoding='utf-8') as f:
    f.write(text.strip())

In [4]:
# Separate Tsakonian and Greek parts
text_list = text.strip().split('\n')
tsakonian_text = [line.split(' ; ')[0] for line in text_list]
greek_text = [line.split(' ; ')[1] for line in text_list]

assert len(tsakonian_text) == len(greek_text), 'Tsakonian and Greek parts must have the same length.'

In [5]:
# Build initial dataframe
temp_df = pd.DataFrame({'tsakonian' : tsakonian_text, 
                        'greek' : greek_text})
temp_df

,tsakonian,greek
0,Το κρίε - Τα κρίατα,Το κρέας - Τα κρέατα
1,Το γα,Το γάλα
2,Α αφόκιουρη,Η μυζήθρα
3,Το άλητε - Τα άλητα,Το αλεύρι - Τα αλεύρια
4,Οι κουλίντζου,Είδος ζυμαρικού
5,Α κολιούρα,Πίτα που ψήνεται στην πλάνη
6,Α γριτσ̇έα - Οι γριτσέλε,Η κουλούρα - Οι κουλούρες
7,Το γιοργούκι,Το γιαούρτι
8,Το άι - Τα άζα,Το λάδι - Τα λάδια
9,Το κάζ̌υ - Τα κάζ̌α,Το καρύδι - Τα καρύδια


## Process Tsakonian column

### Determine gender

In [6]:
# Extract the gender
gender_dict = {
    "Ο" : "Α",
    "Α" : "Θ",
    "To" : "Υ",
    "Το" : "Υ",
    "Τα" : "Υ",
    "Οι" : np.nan,
}

In [7]:
temp_df['article'] = temp_df['tsakonian'].apply(lambda x: x.split(' ')[0])
temp_df['gender'] = temp_df['article'].apply(lambda x: gender_dict[x] if x in gender_dict.keys() else np.nan)
temp_df

,tsakonian,greek,article,gender
0,Το κρίε - Τα κρίατα,Το κρέας - Τα κρέατα,Το,Υ
1,Το γα,Το γάλα,Το,Υ
2,Α αφόκιουρη,Η μυζήθρα,Α,Θ
3,Το άλητε - Τα άλητα,Το αλεύρι - Τα αλεύρια,Το,Υ
4,Οι κουλίντζου,Είδος ζυμαρικού,Οι,NaN
5,Α κολιούρα,Πίτα που ψήνεται στην πλάνη,Α,Θ
6,Α γριτσ̇έα - Οι γριτσέλε,Η κουλούρα - Οι κουλούρες,Α,Θ
7,Το γιοργούκι,Το γιαούρτι,Το,Υ
8,Το άι - Τα άζα,Το λάδι - Τα λάδια,Το,Υ
9,Το κάζ̌υ - Τα κάζ̌α,Το καρύδι - Τα καρύδια,Το,Υ


### Determine plural paradigm

In [8]:
def remove_accents(input_text):
    nfkd_form = unicodedata.normalize('NFKD', input_text)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [9]:
def compute_masculine_paradigm(entry_: pd.Series):
    """
    Computes the hypothesized paradigm for a given masculine entry.
    """

    # Remove accents from the  plural
    entry_['plural'] = remove_accents(entry_['plural'])

    ### Apply rules ###
    # A1: plural in -οι
    if entry_['plural'].endswith('οι'):
        return "Α1"
    
    # A4: plural in -νε 
    # NOTE: plurals are formed by adding -ούνε to the singular
    # but it is checked here as -νε for simplicity
    elif entry_['plural'].endswith('ουνε'):
        return "Α4"
    
    # A5: plural in -δε
    elif entry_['plural'].endswith('δε'):
        return "Α5"
    
    # A2: plural in -ε
    elif entry_['plural'].endswith('ε'):
        return "Α2"
    
    # A3: plural in -ου
    elif entry_['plural'].endswith('ου'):
        return "Α3"
    
    # NaN if no rule applies
    else:
        return "Α"

In [10]:
def compute_femenine_paradigm(entry_: pd.Series):
    """
    Computes the hypothesized paradigm for a given feminine entry.
    """

    # Remove accents from the word
    entry_['singular'] = remove_accents(entry_['singular'])

    ### Apply rules ###
    # Θ3: plural in -άε
    # If it does not hold, remove the accent from the plural suffix
    if entry_['plural'].endswith('άε'):
        return "Θ3"
    else:
        entry_['plural'] = remove_accents(entry_['plural'])

    # Θ2: plural in -λε
    if entry_['plural'].endswith('λε'):
        return "Θ2"    
    
    # Θ4: plural in -δε and word does not end in -ου
    elif entry_['plural'].endswith('δε') and not entry_['singular'].endswith('ου'):
        return "Θ4"

    # Θ5: plural in -δε and word ends in -ου
    elif entry_['plural'].endswith('δε') and entry_['singular'].endswith('ου'):
        return "Θ5"
    
    # Θ1: rest of plurals in ending in -ε
    elif entry_['plural'].endswith('ε'):
        return "Θ1"
        
    # NaN if no rule applies
    else:
        return "Θ"

In [11]:
def compute_neuter_paradigm(entry_):
    """
    Computes the hypothesized paradigm for a given neuter entry.
    """

    # Remove accents from the word and plural
    for key in ['singular', 'plural']:
        if entry_[key] is not np.nan:
            entry_[key] = remove_accents(entry_[key])    

    ### Apply rules ###
    # Υ2: word ends in -μα
    if entry_['singular'].endswith('μα'):
        return "Υ2"
    
    # Υ3: plural ends in -ια
    elif entry_['plural'].endswith('ια'):
        return "Υ3"
    
    # Υ4: word ends in -ι and plural ends in -τα
    elif entry_['singular'].endswith('ι') and entry_['plural'].endswith('ητα'):
        return "Υ4"
    
    # Υ5: word ends in -ε and plural ends in -τα
    elif entry_['singular'].endswith('ε') and entry_['plural'].endswith('ατα'):
        return "Υ5"
    
    # Y1: rest of plurals in -α
    elif entry_['plural'].endswith('α'):
        return "Υ1"
    
    # Υ if no rule applies
    else:
        return 'Υ'

In [12]:
def compute_paradigms(entry: pd.Series):
    """
    Extracts noun paradigms based on the plural suffixes.
    """

    # If the plural key is NaN, return the gender
    if entry['plural'] is np.nan:
        return entry['gender']

    # Create a modifyable copy of the df_dict
    # The values will lose the accents, which is not desired
    # for the final dataframe
    copy_entry = copy.deepcopy(entry)

    # Masculine nouns
    if entry['article'] == 'Ο':
        paradigm = compute_masculine_paradigm(copy_entry)
    
    # Femenine nouns
    elif entry['article'] == 'Α':
        paradigm = compute_femenine_paradigm(copy_entry)

    # Neuter nouns
    elif entry['article'] == 'Το':
        paradigm = compute_neuter_paradigm(copy_entry)

    # NaN if no rule applies
    else:
        paradigm = np.nan

    return paradigm

In [13]:
def compute_masculine_paradigm(row: pd.Series):
    # Apply rules
    # A1: plural in -οι
    if row['plural'].endswith('οι'):
        return "Α1"
    
    # A2: plural in -ε
    elif row['plural'].endswith('ε'):
        return "Α2"
    
    # A3: plural in -ου
    elif row['plural'].endswith('ου'):
        return "Α3"
    
    # A4: plural in -νε
    elif row['plural'].endswith('νε'):
        return "Α4"
    
    # A5: plural in -δε
    elif row['plural'].endswith('δε'):
        return "Α5"
    
    # "Α" if no rule applies
    else:
        return "Α"

In [14]:
# Extract singular and plurals
temp_df['singular'] = temp_df['tsakonian'].apply(lambda x: x.split(' - ')[0].split(' ')[1])
temp_df['plural'] = temp_df['tsakonian'].apply(lambda x: x.split(' - ')[1].split(' ')[1] if ' - ' in x else np.nan)

temp_df

,tsakonian,greek,article,gender,singular,plural
0,Το κρίε - Τα κρίατα,Το κρέας - Τα κρέατα,Το,Υ,κρίε,κρίατα
1,Το γα,Το γάλα,Το,Υ,γα,NaN
2,Α αφόκιουρη,Η μυζήθρα,Α,Θ,αφόκιουρη,NaN
3,Το άλητε - Τα άλητα,Το αλεύρι - Τα αλεύρια,Το,Υ,άλητε,άλητα
4,Οι κουλίντζου,Είδος ζυμαρικού,Οι,NaN,κουλίντζου,NaN
5,Α κολιούρα,Πίτα που ψήνεται στην πλάνη,Α,Θ,κολιούρα,NaN
6,Α γριτσ̇έα - Οι γριτσέλε,Η κουλούρα - Οι κουλούρες,Α,Θ,γριτσ̇έα,γριτσέλε
7,Το γιοργούκι,Το γιαούρτι,Το,Υ,γιοργούκι,NaN
8,Το άι - Τα άζα,Το λάδι - Τα λάδια,Το,Υ,άι,άζα
9,Το κάζ̌υ - Τα κάζ̌α,Το καρύδι - Τα καρύδια,Το,Υ,κάζ̌υ,κάζ̌α


In [15]:
# Compute paradigms
temp_df['paradigm'] = temp_df.apply(compute_paradigms, axis=1)
temp_df

,tsakonian,greek,article,gender,singular,plural,paradigm
0,Το κρίε - Τα κρίατα,Το κρέας - Τα κρέατα,Το,Υ,κρίε,κρίατα,Υ5
1,Το γα,Το γάλα,Το,Υ,γα,NaN,Υ
2,Α αφόκιουρη,Η μυζήθρα,Α,Θ,αφόκιουρη,NaN,Θ
3,Το άλητε - Τα άλητα,Το αλεύρι - Τα αλεύρια,Το,Υ,άλητε,άλητα,Υ1
4,Οι κουλίντζου,Είδος ζυμαρικού,Οι,NaN,κουλίντζου,NaN,NaN
5,Α κολιούρα,Πίτα που ψήνεται στην πλάνη,Α,Θ,κολιούρα,NaN,Θ
6,Α γριτσ̇έα - Οι γριτσέλε,Η κουλούρα - Οι κουλούρες,Α,Θ,γριτσ̇έα,γριτσέλε,Θ2
7,Το γιοργούκι,Το γιαούρτι,Το,Υ,γιοργούκι,NaN,Υ
8,Το άι - Τα άζα,Το λάδι - Τα λάδια,Το,Υ,άι,άζα,Υ1
9,Το κάζ̌υ - Τα κάζ̌α,Το καρύδι - Τα καρύδια,Το,Υ,κάζ̌υ,κάζ̌α,Υ1


## Process Greek column

In [16]:
# Keep only the singular form of the word for Greek
temp_df['greek'] = temp_df['greek'].apply(lambda x: x.split(' - ')[0])

# If the word starts with an article, remove it
articles = ['Ο ', 'Η ', 'Το ', 'Οι ', 'Τα ']
temp_df['greek'] = temp_df['greek'].apply(lambda x: ' '.join(x.split(' ')[1:]) if x.startswith(tuple(articles)) else x)

# Lowercase
temp_df['greek'] = temp_df['greek'].apply(lambda x: x.lower())

temp_df

,tsakonian,greek,article,gender,singular,plural,paradigm
0,Το κρίε - Τα κρίατα,κρέας,Το,Υ,κρίε,κρίατα,Υ5
1,Το γα,γάλα,Το,Υ,γα,NaN,Υ
2,Α αφόκιουρη,μυζήθρα,Α,Θ,αφόκιουρη,NaN,Θ
3,Το άλητε - Τα άλητα,αλεύρι,Το,Υ,άλητε,άλητα,Υ1
4,Οι κουλίντζου,είδος ζυμαρικού,Οι,NaN,κουλίντζου,NaN,NaN
5,Α κολιούρα,πίτα που ψήνεται στην πλάνη,Α,Θ,κολιούρα,NaN,Θ
6,Α γριτσ̇έα - Οι γριτσέλε,κουλούρα,Α,Θ,γριτσ̇έα,γριτσέλε,Θ2
7,Το γιοργούκι,γιαούρτι,Το,Υ,γιοργούκι,NaN,Υ
8,Το άι - Τα άζα,λάδι,Το,Υ,άι,άζα,Υ1
9,Το κάζ̌υ - Τα κάζ̌α,καρύδι,Το,Υ,κάζ̌υ,κάζ̌α,Υ1


## Final processing

In [17]:
# Extract final table
columns = ['singular', 'greek', 'paradigm']
final_table = temp_df[columns].copy()

# Add a 'source' column with value 1
final_table['source'] = 1

final_table

,singular,greek,paradigm,source
0,κρίε,κρέας,Υ5,1
1,γα,γάλα,Υ,1
2,αφόκιουρη,μυζήθρα,Θ,1
3,άλητε,αλεύρι,Υ1,1
4,κουλίντζου,είδος ζυμαρικού,NaN,1
5,κολιούρα,πίτα που ψήνεται στην πλάνη,Θ,1
6,γριτσ̇έα,κουλούρα,Θ2,1
7,γιοργούκι,γιαούρτι,Υ,1
8,άι,λάδι,Υ1,1
9,κάζ̌υ,καρύδι,Υ1,1


In [25]:
# Save the dataframe to an excel
path_ = f'./results/{page}.xlsx'
final_table.to_excel(path_, index=False)
print(f'{page} saved.')

# Open the excel
complete_path = os.path.abspath(path_)
os.startfile(complete_path)

38 saved.
